# Machine Learning Experiment \#1

In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

# Plotly imports
import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

from operator import add

import bisect
import datetime
import math
import random
import time

# Keras imports
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
#from keras.layers.core import Dense, Dropout
from keras.layers import Dense, Dropout, LSTM, Embedding, SimpleRNN

import AlgoPlotting as aplt
import AlgoProcessing as aprc

In [ ]:
datafile = r'''..\Data\IVE_bidaskvol1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose', 'Volume']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [ ]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2018,6,1,0,0,0)].copy()
#pricedata = pricedata[pricedata['DateTime'] < datetime.datetime(2019,1,3,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [ ]:
def AddIndicatorRSI(timeframe, periods=14):
    diffdata = np.diff(pricedata['BidClose'])
    diffdata = np.insert(diffdata,0,0)
    RSI = len(pricedata) * [None]
    
    index = 0
    period = 0
    gains = 0
    losses = 0
    while index < len(pricedata):
        if period < periods:
            period_weight = 1.0/(period+1.0)
        else:
            period_weight = 1.0/periods
            
        period_change = 0
        starttime = pricedata['DateTime'][index]
        while True:
            if index == len(pricedata):
                break
            if pricedata['DateTime'][index] - starttime >= datetime.timedelta(minutes=timeframe):
                break
            
            period_change += diffdata[index]
            if period_change > 0:
                RS_gains = period_change * period_weight + gains * (1.0 - period_weight)
                RS_losses = 0 * period_weight + losses * (1.0 - period_weight)
            else:
                RS_gains = 0 * period_weight + gains * (1.0 - period_weight)
                RS_losses = - period_change * period_weight + losses * (1.0 - period_weight)
                
            if RS_losses == 0:
                RSI[index] = 1.0
            else:
                # Yes, the 2.0 is wrong, but I assume it's best to have NN inputs averaging zero?
                RSI[index] = 1.0 - 2.0/(1.0 + RS_gains/RS_losses)
            
            index += 1
            
        if period_change > 0:
            gains = period_change * period_weight + gains * (1.0 - period_weight)
            losses = 0 * period_weight + losses * (1.0 - period_weight)
        else:
            gains = 0 * period_weight + gains * (1.0 - period_weight)
            losses = - period_change * period_weight + losses * (1.0 - period_weight)
            
        period += 1
        
    pricedata['Rsi' + str(timeframe)] = RSI

In [ ]:
# A bit of a bastardized version of On-Balance Volume...
def AddIndicatorOBV(periods=14, averaging=100):
    diffdata = np.diff(pricedata['BidClose'])
    diffdata = np.insert(diffdata,0,0)
    OBV = len(pricedata) * [None]
    
    avg_vol = 0
    obv = 0
    index = 0
    while index < len(pricedata):
        if index < periods:
            period_weight = 1.0/(index+1.0)
        else:
            period_weight = 1.0/periods
        
        if index < averaging:
            averaging_weight = 1.0/(index+1.0)
        else:
            averaging_weight = 1.0/averaging
            
        period_vol = pricedata['Volume'][index]
        avg_vol = period_vol * averaging_weight + avg_vol * (1.0 - averaging_weight)
            
        period_change = diffdata[index]
        if period_change > 0:
            OBV[index] = period_vol / avg_vol * period_weight + obv * (1.0 - period_weight)
        elif period_change < 0:
            OBV[index] = - period_vol / avg_vol * period_weight + obv * (1.0 - period_weight)
        else:
            OBV[index] = obv * (1.0 - period_weight)
        
        OBV[index] = 3.0/(1.0 + np.exp(-OBV[index]))-1.5
        obv = OBV[index]
        index += 1
            
    pricedata['Obv' + str(periods)] = OBV

In [ ]:
# A bit of a bastardized version of On-Balance Volume...
def AddIndicatorDiff():
    diffdata = np.diff(pricedata['BidClose'])
    diffdata = np.insert(diffdata,0,0)
            
    pricedata['DiffPrice'] = diffdata

In [ ]:
AddIndicatorRSI(1)
AddIndicatorRSI(5)
AddIndicatorRSI(30)
AddIndicatorOBV(14)
AddIndicatorDiff()

In [ ]:
pricedata
xy_chart = aplt.XYChart(y=[pricedata['Volume']/10000, pricedata['Obv14']],
                       names = ['Volume', 'OBV14', 'Diff'])

In [ ]:
# Method that returns a tuple containing numpy arrays of NN inputs and rewards from the specified date range
def get_inputs_and_rewards(start_date, end_date, pricedata):
    inputs = []
    rewards = []
    days = []
    
    #xy_chart = aplt.XYChart()
    
    # Get list of days
    curr_day = start_date
    while curr_day < end_date:
        days.append(curr_day)
        curr_day = curr_day + datetime.timedelta(days=1)

    # Iterate over days and get inputs/rewards
    for day in days:
        daypricedata = pricedata[pricedata['DateTime'] > pd.Timestamp(day)].copy()
        daypricedata = daypricedata[daypricedata['DateTime'] < pd.Timestamp(day) + datetime.timedelta(days=1)].copy()
        daypricedata = daypricedata.reset_index()
        N = 0#agent.inputs + 1
        n = 10
        M = len(daypricedata)-N-n
        
        stock_price = np.array(daypricedata['BidClose'])
        day_rewards_raw = aprc.generate_prediction_data(daypricedata['BidClose'], sell_target_diff=0.2, stop_loss_diff=0.2)
        if day_rewards_raw.shape[0] > 0:
            day_rewards = aprc.filter_prediction_data(day_rewards_raw, tol=3)
            min_val = np.min(stock_price)
            max_val = np.max(stock_price)
            #xy_chart.update(y=[stock_price - min_val, (max_val - min_val)*day_rewards_raw, (max_val - min_val)*day_rewards], names=['stock_price', 'day_rewards_raw', 'day_rewards'])

        for index in range(0,M):
            inputs.append(np.array([daypricedata['Rsi1'][index],
                                    daypricedata['Rsi5'][index],
                                    daypricedata['Obv14'][index]]))
            #rewards.append(daypricedata['BidClose'][index+N+n] - daypricedata['BidClose'][index+N])
            rewards.append(day_rewards[index])
            
            
    return (np.array(inputs), np.array(rewards))

In [ ]:
#get_inputs_and_rewards(datetime.datetime(2019,1,1,0,0,0), datetime.datetime(2019,2,1,0,0,0), pricedata)

In [ ]:
class DQNAgent(object):

    def __init__(self):
        self.inputs = 3
        self.outputs = 1
        self.batch_size = 30
        self.model = self.network()
        print(self.model.summary())
            
    def network(self, weights=None):
        model = Sequential()
        
        num_layers = 3
        neurons_per_layer = 20
        dropout = 0.0
        reg = 0.00000
        act = 'tanh'
        output_act = 'sigmoid'
        learning_rate = 0.0002
        decay_rate = 0.0005
        opt = optimizers.Adam(lr=learning_rate, decay=decay_rate)
        
        if num_layers == 0:
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act, input_dim=self.inputs,
                        kernel_regularizer=regularizers.l2(reg),
                        activity_regularizer=regularizers.l2(reg)))
        else:
            model.add(Dense(units=neurons_per_layer, activation=act, input_dim=self.inputs,
                            kernel_regularizer=regularizers.l2(reg),
                            activity_regularizer=regularizers.l2(reg)))
            
            for ii in range(num_layers - 1):
                if dropout > 0:
                    model.add(Dropout(dropout))
                model.add(Dense(units=neurons_per_layer, activation=act,
                                kernel_regularizer=regularizers.l2(reg),
                                activity_regularizer=regularizers.l2(reg)))
                
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act,
                            kernel_regularizer=regularizers.l2(reg),
                            activity_regularizer=regularizers.l2(reg)))
    
        model.compile(loss='binary_crossentropy', optimizer=opt)
#         model = Sequential()
#         batch_input_shape = (self.batch_size,self.inputs,1)
#         los = 'binary_crossentropy' # 'mse'
#         act = 'tanh'
#         learning_rate = 0.00005*1000
#         decay_rate = learning_rate/1000
#         momentum = 0.0
#         sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
#         opt = sgd
#         kreg = None # regularizers.l2(0.0005)
#         areg = None # regularizers.l2(0.0005)
#         neurons = 3
#         num_recurrent_layers = 3
#         for _ in range(max(num_recurrent_layers - 1, 0)):
#             model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
#                                      kernel_regularizer=kreg, activity_regularizer=areg))
#         model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True,
#                                  kernel_regularizer=kreg, activity_regularizer=areg))
#         model.add(Dense(neurons, activation=act, 
#                              kernel_regularizer=kreg, activity_regularizer=areg))
#         model.add(Dense(1, activation='sigmoid'))

#         model.compile(loss=los, optimizer=opt)

        return model    

    # Train the NN    
    def train(self, inputs, outputs, epochs):
        return self.model.fit(inputs, outputs, epochs=epochs, verbose=0, batch_size=self.batch_size)
    
    # Train and evaluate at intervals on test data to see if the losses on test data are decreasing
    def train_and_evaluate(self, x_train, y_train, x_test, y_test, epochs, epoch_eval_interval):
        train_len = x_train.shape[0] - x_train.shape[0]%self.batch_size
        test_len = x_test.shape[0] - x_test.shape[0]%self.batch_size
        
        x_train = x_train[:train_len]
        y_train = y_train[:train_len]
        x_test = x_test[:test_len]
        y_test = y_test[:test_len]
        
        intervals = int(np.ceil(epochs/epoch_eval_interval))
        testing_losses = []
        testing_losses_idx = []
        training_losses = []
        
        # Initialize charts
        input_chart = aplt.XYChart(title='Input', x_label='Index', y_label='X', names=['Test Input ' + str(ii) for ii in range(x_test.shape[1])])
        predictions_chart = aplt.XYChart(title='Predictions', x_label='Index', y_label='Y', names=['Prediction', 'Test Data'])
        losses_chart = aplt.XYChart(title='Losses', x_label='Epochs', y_label='Loss')
        
        # Reshape
#         s = x_test.shape
#         x_test = x_test.reshape(s[0], s[1], 1)
#         s = x_train.shape
#         x_train = x_train.reshape(s[0], s[1], 1)
        
        # Train in intervals of epochs so we can evaluate the NN performance on testing data in between
        for ii in range(intervals):
            history = self.train(x_train, y_train, epochs=epoch_eval_interval)
            interval_training_losses = history.history['loss']
            self.model.reset_states()
            
            training_losses = training_losses + interval_training_losses
            training_predictions = self.model.predict(x_train, verbose=0, batch_size=self.batch_size)
            self.model.reset_states()
            
            interval_testing_loss = self.model.evaluate(x_test, y_test, verbose=0, batch_size=self.batch_size)
            self.model.reset_states()
            predictions = self.model.predict(x_test, verbose=0, batch_size=self.batch_size)
            testing_losses.append(interval_testing_loss)
            testing_losses_idx.append((ii+1)*epoch_eval_interval)
            self.model.reset_states()

            input_chart.update(y=[x_test[:,ii] for ii in range(x_test.shape[1])])
            predictions_chart.update(y=[training_predictions, y_train])
            losses_chart.update(y=[testing_losses, training_losses],
                                x=[[epoch_eval_interval*ii for ii in range(len(testing_losses))], [ii for ii in range(len(training_losses))]],
                                names=['Testing Losses', 'Training Losses'])
        
        # Convert to np.array
        testing_losses = np.array(testing_losses)
        testing_losses_idx = np.array(testing_losses_idx)
        
        return (training_losses, testing_losses, testing_losses_idx)
            

In [ ]:
agent = DQNAgent()

# Indexs for training and testing ranges
training_start_idx = 300
training_end_idx = np.floor(len(pricedata)/2)
testing_start_idx = training_end_idx + 1
testing_end_idx = len(pricedata) -  1

# Training date range
training_start_date = pricedata['DateTime'][training_start_idx].date()
training_end_date = pricedata['DateTime'][training_end_idx].date()

# Testing date range
testing_start_date = pricedata['DateTime'][testing_start_idx].date()
testing_end_date = pricedata['DateTime'][testing_end_idx].date()

# Training inputs and rewards
(x_train, y_train) = get_inputs_and_rewards(start_date=training_start_date,
                                                             end_date=training_end_date,
                                                             pricedata=pricedata)
# Testing inputs and rewards
(x_test, y_test) = get_inputs_and_rewards(start_date=testing_start_date,
                                                           end_date=testing_end_date,
                                                           pricedata=pricedata)


In [ ]:
# Amount of training time
epochs = 10000
epoch_eval_interval = 1

# Train the NN
(training_losses, testing_losses, testing_losses_idx) = agent.train_and_evaluate(x_train=x_train,
                                                                                 y_train=y_train,
                                                                                 x_test=x_test,
                                                                                 y_test=y_test,
                                                                                 epochs=epochs,
                                                                                 epoch_eval_interval=epoch_eval_interval)

# Get the performance on the training and testing data after training
training_predictions = agent.model.predict_on_batch(x_train)
testing_predictions = agent.model.predict_on_batch(x_test)

In [ ]:
training_idx_plot = [ii for ii in range(len(training_predictions))]
testing_idx_plot = [ii for ii in range(len(testing_predictions))]

training_predictions_plot = [w[0] for w in training_predictions]
testing_predictions_plot = [w[0] for w in testing_predictions]

print('Mean Training Prediction: ', np.mean(training_predictions_plot))
print('Mean Training Reward:     ', np.mean(training_rewards))
print('Mean Testing Prediction:  ', np.mean(testing_predictions_plot))
print('Mean Testing Reward:      ', np.mean(testing_rewards))

In [ ]:
testing_predictions_chart = PredictionsChart(testing_idx_plot, testing_rewards, testing_predictions_plot, title='Testing')
testing_predictions_chart.display

# To-Do List

* Extract volume per minute data from tick data
* LossesChart/etc. applied to separate test data
* Try feeding a vector of indicators instead of raw price data
* Implement actual buy/sell strategy based on NN output, evaluate performance